In [4]:
import os.path
from os.path import join
import numpy as np
from os import listdir
from os.path import join
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from plotly import graph_objs as go
%matplotlib inline
import datetime as dt
import time
import yfinance as yf

In [31]:
# Devuelve información importante sobre la acción cuyo ticker se pase como
# parametro
def get_info_accion(ticker):
    accion=yf.Ticker(ticker)
    #hist = accion.history(period="max")["Close"]
    print(accion.info)

# Crea un archivo .csv con la información diaria sobre el activo
# cuyo ticker se pase como parametro
def guardar_accion_csv(carpeta,ticker):
    accion = yf.Ticker(ticker)
    try:
        print("Guardando información de:",ticker)
        df = accion.history(period="5y")
        time.sleep(2)
        if df.empty:
            acciones_sin_descargar.append(ticker)
        archivo = carpeta + ticker.replace(".","_")+".csv"
        print(archivo)
        df.to_csv(archivo)
    except Exception as ex:
        acciones_sin_descargar.append(ticker)
        print("No se ha podido descargar",ticker)

# Devuelve un DF de la accion correspondiente al ticker
def get_df_desde_csv(carpeta,ticker):
    return pd.read_csv(carpeta+ticker+".csv", index_col=0)

def set_retornos_diarios_al_df(df):
    df['retorno diario']= (df['Close']/df['Close'].shift(1)) - 1
    return df

def set_retornos_acumulados_al_df(df):
    df["retorno acumulado"]= (1 + df["retorno diario"]).cumprod()
    return df

def set_bandas_bollinger_al_df(df):
    df["banda_medio"] = df['Close'].rolling(window=20).mean()
    df["banda_superior"] = df['banda_medio'] + 1.96 * df['Close'].rolling(window=20).std()
    df["banda_inferior"] = df['banda_medio'] - 1.96 * df['Close'].rolling(window=20).std()
    return df


def set_ichimoku_al_df(df):
    #Linea de Conversión
    alto = df["High"].rolling(window=9).max()
    bajo = df["Low"].rolling(window=9).min()
    df["Conversion"] = (alto+bajo)/2
    #Linea Base
    alto = df["High"].rolling(window=26).max()
    bajo = df["Low"].rolling(window=26).min()
    df["Base"] = (alto+bajo)/2
    #Span A
    df["SpanA"]=((df["Conversion"] + df["Base"])/2)
    #Span B
    alto = df["High"].rolling(window=52).max()
    bajo = df["Low"].rolling(window=52).min()
    df["SpanB"]=((alto + bajo)/2).shift(26)
    #Lagging Span
    df["Lagging"] = df["Close"].shift(-26)

    return df

def set_info_df(df):
    set_retornos_diarios_al_df(df)
    set_retornos_acumulados_al_df(df)
    set_bandas_bollinger_al_df(df)
    set_ichimoku_al_df(df)

    return df

def actualizar_info_stocks(path):
    archivos = [x for x in listdir(path) if os.path.isfile(join(path,x))]
    tickers = [os.path.splitext(x)[0] for x in archivos]
    numero_tickers = len(tickers)

    for i in range(numero_tickers):
        try:
            print("Editando "+tickers[i]+"...")
            df = get_df_desde_csv(path,tickers[i])
            set_info_df(df)
            df.to_csv(path+tickers[i]+".csv")
        except Exception as ex:
            print("No se ha podido editar:",tickers[i])

def plot_candlesticks(df):
    fig = go.Figure()
    candle = go.Candlestick(x=df.index, open = df['Open'],high=df['High'],low=df['Low'],close=df["Close"],name="Candlestick")
    fig.add_trace(candle)
    fig.update_xaxes(title="Fecha",rangeslider_visible=True)
    fig.update_xaxes(title="Precio")
    fig.show()

def plot_bandas_bollinger(df,ticker):
    fig = go.Figure()

    candle = go.Candlestick(x=df.index, open=df['Open'],high=df['High'], low=df['Low'],close=df['Close'], name="Candlestick")
    upper_line = go.Scatter(x=df.index, y=df['banda_superior'],line=dict(color='rgba(250, 0, 0, 0.75)',width=1), name="Upper Band")
    mid_line = go.Scatter(x=df.index, y=df['banda_medio'],line=dict(color='rgba(0, 0, 250, 0.75)',width=0.7), name="Middle Band")
    lower_line = go.Scatter(x=df.index, y=df['banda_inferior'],line=dict(color='rgba(0, 250, 0, 0.75)',width=1), name="Lower Band")

    fig.add_trace(candle)
    fig.add_trace(upper_line)
    fig.add_trace(mid_line)
    fig.add_trace(lower_line)

    fig.update_xaxes(title="Fecha", rangeslider_visible=True)
    fig.update_yaxes(title="Precio")

    fig.update_layout(title=ticker + " Bandas Bollinger",height=1200, width=1800, showlegend=True)
    fig.show()

def color_accion(marca):
    if marca >=1:
        return 'rgba(0,250,0,0.3)'
    else:
        return 'rgba(250,0,0,0.3)'

def plot_ichimoku(df,ticker):
    fig = go.Figure()

    df_copia=df.copy()

    df['marca'] = np.where(df['SpanA']>df['SpanB'],1,0)
    df['group'] = df['marca'].ne(df['marca'].shift()).cumsum()
    df = df.groupby('group')

    dfs =[]
    for name,data in df:
        dfs.append(data)
    for df in dfs:
        fig.add_traces(go.Scatter(x=df.index,y=df['SpanA'],line=dict(color='rgba(0,0,0,0)')))
        fig.add_traces(go.Scatter(x=df.index,y=df['SpanB'],line=dict(color='rgba(0,0,0,0)'),
                                 fill='tonexty',fillcolor=color_accion(df['marca'].iloc[0])))

    candle = go.Candlestick(x=df_copia.index, open=df_copia['Open'],high=df_copia['High'], low=df_copia['Low'],close=df_copia['Close'], name="Candlestick")
    base = go.Scatter(x=df_copia.index, y=df_copia['Base'],
                      line=dict(color='pink',width=2), name="Base")
    conversion = go.Scatter(x=df_copia.index, y=df_copia['Conversion'],
                            line=dict(color='black',width=1), name="Conversión")
    span_a = go.Scatter(x=df_copia.index, y=df_copia['SpanA'],
                        line=dict(color='green',width=1), name="SpanA")
    span_b = go.Scatter(x=df_copia.index, y=df_copia['SpanB'],
                        line=dict(color='red',width=1), name="SpanB")
    lagging = go.Scatter(x=df_copia.index, y=df_copia['Lagging'],
                         line=dict(color='purple',width=1), name="Lagging")

    fig.add_trace(candle)
    fig.add_trace(conversion)
    fig.add_trace(base)
    fig.add_trace(span_a)
    fig.add_trace(span_b)
    fig.add_trace(lagging)

    fig.update_xaxes(title="Fecha", rangeslider_visible=True)
    fig.update_yaxes(title="Precio")

    fig.update_layout(title=ticker + " Ichimoku",height=1000, width=1500, showlegend=True)
    fig.show()

In [32]:
acciones_sin_descargar = []
path = "./datos/"

In [33]:
actualizar_info_stocks(path)

Editando ACS_MC...
Editando ACX_MC...
Editando AENA_MC...
Editando ALM_MC...
Editando AMD...
Editando AMS_MC...
Editando ANA_MC...
Editando BBVA_MC...
Editando BKT_MC...
Editando CABK_MC...
Editando CIE_MC...
Editando CLNX_MC...
Editando COL_MC...
Editando ELE_MC...
Editando ENG_MC...
Editando FDR_MC...
Editando FER_MC...
Editando GRF_MC...
Editando IAG_MC...
Editando IBE_MC...
Editando IDR_MC...
Editando ITX_MC...
Editando MAP_MC...
Editando MEL_MC...
Editando MRL_MC...
Editando MTS_MC...
Editando NTGY_MC...
Editando PHM_MC...
Editando REE_MC...
Editando REP_MC...
Editando ROVI_MC...
Editando SAB_MC...
Editando SAN_MC...
Editando SGRE_MC...
Editando SLR_MC...
Editando TEF_MC...


In [34]:
df = get_df_desde_csv(path,"TEF_MC")
plot_ichimoku(df,"TEF_MC")